In [1]:
import numpy as np
import psycopg2
import datetime
import pandas as pd

In [113]:
#計測地点のosm_idを指定
start_osm = 315917801
mid_osm = []
end_osm = 315499982
start_mesh = 50325623342
end_mesh = 50325613134

In [114]:
connection = psycopg2.connectconnection = psycopg2.connect(host="localhost", database="analysisdb", user="postgres",password="setouchi1311")

In [115]:

sttime = "2022-04-01 12:00:00"
dbname = "kddi_trkn12_change_f3_23"

In [116]:
datedir = "230213"
f = open(datedir+'/'+ dbname + '.csv', 'w')

cur =connection.cursor()
sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(start_mesh)+"'"
cur.execute(sql)
for row in cur:
    stlat = row[0]
    stlng = row[1]
sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(end_mesh)+"'"
cur.execute(sql)
for row in cur:
    edlat = row[0]
    edlng = row[1]
cur.close()

if stlat > edlat:
    wklat = stlat
    stlat = edlat
    edlat = wklat
if stlng > edlng:
    wklng = stlng
    stlng = edlng
    edlng = wklng
            
for i in range(30):
    dte = datetime.datetime.strptime(sttime, '%Y-%m-%d %H:%M:%S')
    if i != 0:
        dte = dte + datetime.timedelta(days=i)
    for j in range(24):
        #特定日付のKDDIメッシュ情報を全取得
        cur =connection.cursor()
        sql = "SELECT mesh, tbran,pointdata.latitude,pointdata.longitude FROM kddi_12_param  "
        sql += "INNER JOIN pointdata ON mesh = cast(pointdata.grid_code AS BIGINT)  "
        sql += "WHERE times = '"+str(dte)+"' AND pointdata.grid_code != '0'  ORDER BY mesh "
        cur.execute(sql)
        kddiDf = pd.DataFrame.from_records(cur.fetchall(), columns=["mesh","tbran","latitude","longitude"])
        cur.close()

        one_target_list = []
        one_target_list.append(start_mesh)
        one_target_list.append(end_mesh)
        two_target_list = []
        two_target_list.append(start_osm)
        if len(mid_osm) != 0:
            for target in mid_osm:
                two_target_list.append(target)
        if start_osm != end_osm:
            two_target_list.append(end_osm)


        print(str(stlat) + ":" + str(edlat) + ":" + str(stlng) + ":" + str(stlng))

        keylist = []
        cnt = 0
        for item in two_target_list:
            sql = "SELECT a.grid_code,a.latitude,a.longitude"
            sql += " FROM pointdata a, roadmap r"
            sql += " WHERE osm_id = "+ str(item) +" and ST_Distance(a.geom, r.geom) < 0.001"
            sql += " order by ST_Distance(a.geom, r.geom);"
            cur =connection.cursor()
            cur.execute(sql)
            childlist = []
            for row in cur:
                if row[1] >= stlat and row[1] <= edlat and row[2] >= stlng and row[2] <= edlng:
                    childlist.append([row[0], row[1], row[2]])
                #print(row[0])
            worklist = [item, childlist]
            keylist.append(worklist)
            cnt = cnt + 1
            cur.close()
        #特定日付のトラカンデータを全取得 > 緯度経度
        cur =connection.cursor()
        sql = "SELECT trakanmap.gridcode , 断面交通量,pointdata.latitude,pointdata.longitude FROM trakan2204 "
        sql += "INNER JOIN trakanmap ON trakan2204.計測地点番号 = trakanmap.地点番号 "
        sql += "INNER JOIN pointdata ON trakanmap.gridcode = pointdata.grid_code "
        sql += "WHERE 時刻 = '"+str(dte)+"' AND (trakanmap.gridcode = '"+str(start_mesh)+"' or trakanmap.gridcode = '"+str(end_mesh)+"') "
        sql += "ORDER BY trakanmap.gridcode"
        cur.execute(sql)
        trknDf = pd.DataFrame(cur.fetchall(), columns=["gridcode","danmen","latitude","longitude"])
        cur.close()
        #trknDf
        #keylist

        from geopy.distance import geodesic

        #データが存在する道路IDごとに交通量を平均化して取得する
        lstItems = []
        cntDt = 0
        wks = ""
        wknp=""
        find = 0

        trknStParam = trknDf.query('gridcode == "'+str(start_mesh) + '"')
        trknEdParam = trknDf.query('gridcode == "'+str(end_mesh) + '"')

        if len(trknStParam) == 0:
            print(str(start_mesh) + " mesh empty!")
            continue
        elif len(trknStParam) == 1:
            trknSt = trknStParam['danmen'].iloc[-1]
        else:
            trknSt = trknStParam['danmen'].iloc[0] + trknStParam['danmen'].iloc[1]

        if len(trknEdParam) == 0:
            print(str(end_mesh) + " mesh empty!")
            continue
        elif len(trknEdParam) == 1:
            trknEd = trknEdParam['danmen'].iloc[-1]
        else:
            trknEd = trknEdParam['danmen'].iloc[0] + trknEdParam['danmen'].iloc[1]

        st_station = [trknStParam['latitude'].iloc[-1], trknStParam['longitude'].iloc[-1]]
        ed_station = [trknEdParam['latitude'].iloc[-1], trknEdParam['longitude'].iloc[-1]]

        dis_km = geodesic(st_station, ed_station)
        cur =connection.cursor()
        for row in keylist:
            for meshid in row[1]:
                #路線のメッシュ数分ループ
                print(str(meshid[0]))
                kddiParam = kddiDf.query('mesh == '+str(meshid[0]))
                if len(kddiParam) == 0:
                    print(str(meshid[0]) + " is kddi none!")
                    continue
                #路線のメッシュがkddiに存在したなら
                
                kddi_station = [kddiParam['latitude'].iloc[-1], kddiParam['longitude'].iloc[-1]]
                kddi_km = geodesic(st_station, kddi_station)
                
                if trknSt < trknEd:
                    ret = ((trknEd - trknSt) * (kddi_km / dis_km)) + trknSt
                else:
                    ret = trknSt - ((trknEd - trknSt) * (kddi_km / dis_km))
                print("存在メッシュ：" + str(meshid[0]) + "値" + str(ret) + "距離" + str(kddi_km))
                sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + ") ON CONFLICT DO NOTHING;"
                #cur.execute(sql)
                f.write(str(dte) +  "," + str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + '\n')
                #connection.commit()

        sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(start_mesh) + ","  + str(trknSt) + ") ON CONFLICT DO NOTHING;"
        f.write(str(dte) +  "," + str(start_mesh)+","  +str(end_mesh)+","  + str(start_mesh) + ","  + str(trknSt) + '\n')
        #cur.execute(sql)
        #connection.commit()
                
        cur.close()
        #print(lstItems)
        dte = dte + datetime.timedelta(minutes=15)
f.close()

33.7619791665:33.7734375000:132.789843750:132.789843750
50325613334
存在メッシュ：50325613334値26.0距離0.8590233749368513 km
50325623342
存在メッシュ：50325623342値26.0距離0.0 km
50325623321
存在メッシュ：50325623321値26.0距離0.2726648230002139 km
50325623114
存在メッシュ：50325623114値26.0距離0.6461675098581355 km
50325623132
存在メッシュ：50325623132値26.0距離0.5453313922032513 km
50325623112
存在メッシュ：50325623112値26.0距離0.7512505454326793 km
50325623324
存在メッシュ：50325623324値26.0距離0.11553986804341464 km
50325623134
存在メッシュ：50325623134値26.0距離0.4515962139681325 km
50325623143
存在メッシュ：50325623143値26.0距離0.3756243415432246 km
50325623323
存在メッシュ：50325623323値26.0距離0.1851957134200454 km
50325613332
存在メッシュ：50325613332値26.0距離0.9685885967790505 km
50325613312
存在メッシュ：50325613312値26.0距離1.1911110587401013 km
50325613314
存在メッシュ：50325613314値26.0距離1.07940043987984 km
50325613134
存在メッシュ：50325613134値26.0距離1.303489444211249 km
50325613334
存在メッシュ：50325613334値26.0距離0.8590233749368513 km
50325613332
存在メッシュ：50325613332値26.0距離0.9685885967790505 km
33.7619791665:33.

33.7619791665:33.7734375000:132.789843750:132.789843750
50325613334
存在メッシュ：50325613334値33.0距離0.8590233749368513 km
50325623342
存在メッシュ：50325623342値33.0距離0.0 km
50325623321
存在メッシュ：50325623321値33.0距離0.2726648230002139 km
50325623114
50325623114 is kddi none!
50325623132
存在メッシュ：50325623132値33.0距離0.5453313922032513 km
50325623112
存在メッシュ：50325623112値33.0距離0.7512505454326793 km
50325623324
存在メッシュ：50325623324値33.0距離0.11553986804341464 km
50325623134
存在メッシュ：50325623134値33.0距離0.4515962139681325 km
50325623143
存在メッシュ：50325623143値33.0距離0.3756243415432246 km
50325623323
50325623323 is kddi none!
50325613332
存在メッシュ：50325613332値33.0距離0.9685885967790505 km
50325613312
存在メッシュ：50325613312値33.0距離1.1911110587401013 km
50325613314
存在メッシュ：50325613314値33.0距離1.07940043987984 km
50325613134
存在メッシュ：50325613134値33.0距離1.303489444211249 km
50325613334
存在メッシュ：50325613334値33.0距離0.8590233749368513 km
50325613332
存在メッシュ：50325613332値33.0距離0.9685885967790505 km
33.7619791665:33.7734375000:132.789843750:132.789843750
503

33.7619791665:33.7734375000:132.789843750:132.789843750
50325613334
存在メッシュ：50325613334値29.0距離0.8590233749368513 km
50325623342
存在メッシュ：50325623342値29.0距離0.0 km
50325623321
存在メッシュ：50325623321値29.0距離0.2726648230002139 km
50325623114
存在メッシュ：50325623114値29.0距離0.6461675098581355 km
50325623132
存在メッシュ：50325623132値29.0距離0.5453313922032513 km
50325623112
存在メッシュ：50325623112値29.0距離0.7512505454326793 km
50325623324
存在メッシュ：50325623324値29.0距離0.11553986804341464 km
50325623134
存在メッシュ：50325623134値29.0距離0.4515962139681325 km
50325623143
存在メッシュ：50325623143値29.0距離0.3756243415432246 km
50325623323
50325623323 is kddi none!
50325613332
存在メッシュ：50325613332値29.0距離0.9685885967790505 km
50325613312
50325613312 is kddi none!
50325613314
存在メッシュ：50325613314値29.0距離1.07940043987984 km
50325613134
存在メッシュ：50325613134値29.0距離1.303489444211249 km
50325613334
存在メッシュ：50325613334値29.0距離0.8590233749368513 km
50325613332
存在メッシュ：50325613332値29.0距離0.9685885967790505 km
33.7619791665:33.7734375000:132.789843750:132.789843750
503

33.7619791665:33.7734375000:132.789843750:132.789843750
50325613334
存在メッシュ：50325613334値34.0距離0.8590233749368513 km
50325623342
存在メッシュ：50325623342値34.0距離0.0 km
50325623321
50325623321 is kddi none!
50325623114
50325623114 is kddi none!
50325623132
50325623132 is kddi none!
50325623112
50325623112 is kddi none!
50325623324
存在メッシュ：50325623324値34.0距離0.11553986804341464 km
50325623134
存在メッシュ：50325623134値34.0距離0.4515962139681325 km
50325623143
50325623143 is kddi none!
50325623323
存在メッシュ：50325623323値34.0距離0.1851957134200454 km
50325613332
存在メッシュ：50325613332値34.0距離0.9685885967790505 km
50325613312
50325613312 is kddi none!
50325613314
存在メッシュ：50325613314値34.0距離1.07940043987984 km
50325613134
存在メッシュ：50325613134値34.0距離1.303489444211249 km
50325613334
存在メッシュ：50325613334値34.0距離0.8590233749368513 km
50325613332
存在メッシュ：50325613332値34.0距離0.9685885967790505 km
33.7619791665:33.7734375000:132.789843750:132.789843750
50325613334
存在メッシュ：50325613334値35.0距離0.8590233749368513 km
50325623342
存在メッシュ：503256233

33.7619791665:33.7734375000:132.789843750:132.789843750
50325613334
存在メッシュ：50325613334値31.0距離0.8590233749368513 km
50325623342
存在メッシュ：50325623342値31.0距離0.0 km
50325623321
存在メッシュ：50325623321値31.0距離0.2726648230002139 km
50325623114
存在メッシュ：50325623114値31.0距離0.6461675098581355 km
50325623132
存在メッシュ：50325623132値31.0距離0.5453313922032513 km
50325623112
存在メッシュ：50325623112値31.0距離0.7512505454326793 km
50325623324
50325623324 is kddi none!
50325623134
存在メッシュ：50325623134値31.0距離0.4515962139681325 km
50325623143
存在メッシュ：50325623143値31.0距離0.3756243415432246 km
50325623323
50325623323 is kddi none!
50325613332
存在メッシュ：50325613332値31.0距離0.9685885967790505 km
50325613312
50325613312 is kddi none!
50325613314
50325613314 is kddi none!
50325613134
存在メッシュ：50325613134値31.0距離1.303489444211249 km
50325613334
存在メッシュ：50325613334値31.0距離0.8590233749368513 km
50325613332
存在メッシュ：50325613332値31.0距離0.9685885967790505 km
33.7619791665:33.7734375000:132.789843750:132.789843750
50325613334
存在メッシュ：50325613334値26.0距離0.85902

...

33.7619791665:33.7734375000:132.789843750:132.789843750
50325613334
存在メッシュ：50325613334値25.0距離0.8590233749368513 km
50325623342
存在メッシュ：50325623342値25.0距離0.0 km
50325623321
存在メッシュ：50325623321値25.0距離0.2726648230002139 km
50325623114
存在メッシュ：50325623114値25.0距離0.6461675098581355 km
50325623132
存在メッシュ：50325623132値25.0距離0.5453313922032513 km
50325623112
存在メッシュ：50325623112値25.0距離0.7512505454326793 km
50325623324
存在メッシュ：50325623324値25.0距離0.11553986804341464 km
50325623134
50325623134 is kddi none!
50325623143
存在メッシュ：50325623143値25.0距離0.3756243415432246 km
50325623323
存在メッシュ：50325623323値25.0距離0.1851957134200454 km
50325613332
存在メッシュ：50325613332値25.0距離0.9685885967790505 km
50325613312
存在メッシュ：50325613312値25.0距離1.1911110587401013 km
50325613314
存在メッシュ：50325613314値25.0距離1.07940043987984 km
50325613134
50325613134 is kddi none!
50325613334
存在メッシュ：50325613334値25.0距離0.8590233749368513 km
50325613332
存在メッシュ：50325613332値25.0距離0.9685885967790505 km
33.7619791665:33.7734375000:132.789843750:132.789843750
50

33.7619791665:33.7734375000:132.789843750:132.789843750
50325613334
存在メッシュ：50325613334値29.0距離0.8590233749368513 km
50325623342
存在メッシュ：50325623342値29.0距離0.0 km
50325623321
存在メッシュ：50325623321値29.0距離0.2726648230002139 km
50325623114
存在メッシュ：50325623114値29.0距離0.6461675098581355 km
50325623132
存在メッシュ：50325623132値29.0距離0.5453313922032513 km
50325623112
存在メッシュ：50325623112値29.0距離0.7512505454326793 km
50325623324
存在メッシュ：50325623324値29.0距離0.11553986804341464 km
50325623134
存在メッシュ：50325623134値29.0距離0.4515962139681325 km
50325623143
存在メッシュ：50325623143値29.0距離0.3756243415432246 km
50325623323
存在メッシュ：50325623323値29.0距離0.1851957134200454 km
50325613332
存在メッシュ：50325613332値29.0距離0.9685885967790505 km
50325613312
存在メッシュ：50325613312値29.0距離1.1911110587401013 km
50325613314
存在メッシュ：50325613314値29.0距離1.07940043987984 km
50325613134
50325613134 is kddi none!
50325613334
存在メッシュ：50325613334値29.0距離0.8590233749368513 km
50325613332
存在メッシュ：50325613332値29.0距離0.9685885967790505 km
33.7619791665:33.7734375000:132.78984

33.7619791665:33.7734375000:132.789843750:132.789843750
50325613334
存在メッシュ：50325613334値29.0距離0.8590233749368513 km
50325623342
存在メッシュ：50325623342値29.0距離0.0 km
50325623321
存在メッシュ：50325623321値29.0距離0.2726648230002139 km
50325623114
存在メッシュ：50325623114値29.0距離0.6461675098581355 km
50325623132
存在メッシュ：50325623132値29.0距離0.5453313922032513 km
50325623112
存在メッシュ：50325623112値29.0距離0.7512505454326793 km
50325623324
存在メッシュ：50325623324値29.0距離0.11553986804341464 km
50325623134
50325623134 is kddi none!
50325623143
50325623143 is kddi none!
50325623323
存在メッシュ：50325623323値29.0距離0.1851957134200454 km
50325613332
存在メッシュ：50325613332値29.0距離0.9685885967790505 km
50325613312
50325613312 is kddi none!
50325613314
存在メッシュ：50325613314値29.0距離1.07940043987984 km
50325613134
50325613134 is kddi none!
50325613334
存在メッシュ：50325613334値29.0距離0.8590233749368513 km
50325613332
存在メッシュ：50325613332値29.0距離0.9685885967790505 km
33.7619791665:33.7734375000:132.789843750:132.789843750
50325613334
存在メッシュ：50325613334値37.0距離0.85902

33.7619791665:33.7734375000:132.789843750:132.789843750
50325613334
存在メッシュ：50325613334値21.0距離0.8590233749368513 km
50325623342
存在メッシュ：50325623342値21.0距離0.0 km
50325623321
存在メッシュ：50325623321値21.0距離0.2726648230002139 km
50325623114
存在メッシュ：50325623114値21.0距離0.6461675098581355 km
50325623132
存在メッシュ：50325623132値21.0距離0.5453313922032513 km
50325623112
存在メッシュ：50325623112値21.0距離0.7512505454326793 km
50325623324
存在メッシュ：50325623324値21.0距離0.11553986804341464 km
50325623134
存在メッシュ：50325623134値21.0距離0.4515962139681325 km
50325623143
存在メッシュ：50325623143値21.0距離0.3756243415432246 km
50325623323
存在メッシュ：50325623323値21.0距離0.1851957134200454 km
50325613332
存在メッシュ：50325613332値21.0距離0.9685885967790505 km
50325613312
存在メッシュ：50325613312値21.0距離1.1911110587401013 km
50325613314
存在メッシュ：50325613314値21.0距離1.07940043987984 km
50325613134
存在メッシュ：50325613134値21.0距離1.303489444211249 km
50325613334
存在メッシュ：50325613334値21.0距離0.8590233749368513 km
50325613332
存在メッシュ：50325613332値21.0距離0.9685885967790505 km
33.7619791665:33.

33.7619791665:33.7734375000:132.789843750:132.789843750
50325613334
存在メッシュ：50325613334値29.0距離0.8590233749368513 km
50325623342
存在メッシュ：50325623342値29.0距離0.0 km
50325623321
存在メッシュ：50325623321値29.0距離0.2726648230002139 km
50325623114
存在メッシュ：50325623114値29.0距離0.6461675098581355 km
50325623132
存在メッシュ：50325623132値29.0距離0.5453313922032513 km
50325623112
存在メッシュ：50325623112値29.0距離0.7512505454326793 km
50325623324
存在メッシュ：50325623324値29.0距離0.11553986804341464 km
50325623134
存在メッシュ：50325623134値29.0距離0.4515962139681325 km
50325623143
存在メッシュ：50325623143値29.0距離0.3756243415432246 km
50325623323
存在メッシュ：50325623323値29.0距離0.1851957134200454 km
50325613332
存在メッシュ：50325613332値29.0距離0.9685885967790505 km
50325613312
存在メッシュ：50325613312値29.0距離1.1911110587401013 km
50325613314
存在メッシュ：50325613314値29.0距離1.07940043987984 km
50325613134
50325613134 is kddi none!
50325613334
存在メッシュ：50325613334値29.0距離0.8590233749368513 km
50325613332
存在メッシュ：50325613332値29.0距離0.9685885967790505 km
33.7619791665:33.7734375000:132.78984

In [117]:
connection.close()

In [82]:
#CreateGeojsonFile